# [106] Special subset sums: meta-testing
From the statement, we are given that:
1. S(B) ≠ S(C); that is, sums of subsets cannot be equal.
2. If B contains more elements than C then S(B) > S(C).

We are also given that 2 is satisfied. This means, we only need to be checking for sets of equal size.

### Configuration

Before we begin, this is the boring configuration part:

In [1]:
import logging
import sys

from solutions.euler.util.decorators import timed_function

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

## Example 1
Let's go with the example of 4 -

In [3]:
def compare_4(a: list):
	logging.debug(f'Original = {a}')

	comparison = 0

	subset_b = {a[0], a[3]}
	subset_c = {a[1], a[2]}
	comparison += 1

	return sum(subset_b) == sum(subset_c)


logging.info('Simple example of 4')
assert compare_4([1, 2, 3, 4])
assert not compare_4([1, 2, 3, 100])
logging.info('Complete')

INFO:root:Simple example of 4
DEBUG:root:Original = [1, 2, 3, 4]
DEBUG:root:Original = [1, 2, 3, 100]
INFO:root:Complete


If we sort the list, then we don't have to check b={a[0], a[1]} and c={a[2], a[3]}. a[0] < a[2] and a[1] < a[3] (by definition of sorted array)
It is the same for the configuration b={a[0], a[2]}, c={a[1], a[3]} (a[0] < a[1], a[2] < a[3]). However, we do need to check the configuration {a[0], a[4]} and {a[1], a[2]} because in both sets, there is an element that is bigger and smaller than the other set.

## Intuition
So how about adding every element of the list to either subset b or c, and only if there is an element that is both bigger and smaller in both sets, do we proceed with making a comparison?

In [8]:
def first_attempt(a: list):
	a.sort()

	max_len = len(a) // 2
	comparison = 0

	def helper(b, c, i):
		nonlocal max_len, comparison
		if len(b) < max_len:
			helper(b + [a[i]], c, i + 1)

		if len(c) < max_len:
			helper(b, c + [a[i]], i + 1)

		if len(b) == max_len and len(c) == max_len:
			# I wish there was a simpler way of doing this
			smaller_b, smaller_c = False, False
			for e1, e2 in zip(b, c):
				if e1 > e2: smaller_b = True
				if e1 < e2: smaller_c = True

			if smaller_b and smaller_c:
				logging.debug(f'Comparing::{b} and {c}')
				comparison += 1

	helper(b=[], c=[], i=0)


first_attempt(list(range(1, 8)))

DEBUG:root:Comparing::[1, 2, 6] and [3, 4, 5]
DEBUG:root:Comparing::[1, 3, 6] and [2, 4, 5]
DEBUG:root:Comparing::[1, 4, 5] and [2, 3, 6]
DEBUG:root:Comparing::[1, 4, 6] and [2, 3, 5]
DEBUG:root:Comparing::[1, 5, 6] and [2, 3, 4]
DEBUG:root:Comparing::[2, 3, 4] and [1, 5, 6]
DEBUG:root:Comparing::[2, 3, 5] and [1, 4, 6]
DEBUG:root:Comparing::[2, 3, 6] and [1, 4, 5]
DEBUG:root:Comparing::[2, 4, 5] and [1, 3, 6]
DEBUG:root:Comparing::[3, 4, 5] and [1, 2, 6]


If you were code-reviewing this code, what adjustments would you make?

In [ ]:
def better_q106(a: list):
	a.sort()

	max_len = len(a) // 2
	b, c = [], []
	comparison = 0

	def helper(i=0):
		if 1 < len(b) == len(c):
			big_small = set(e1 < e2 for e1, e2 in zip(b, c))
			if big_small == {False, True}:
				nonlocal comparison
				logging.debug(f'Comparing::{b} and {c}')
				comparison += 1

		if len(b) < max_len:
			for j in range(i, len(a)):
				b.append(a[j])
				helper(j + 1)
				b.pop()

		if len(c) < max_len:
			for j in range(i, len(a)):
				c.append(a[j])
				helper(j + 1)
				c.pop()

	helper()
	return comparison // 2


assert (timed_function(better_q106)((list(range(1, 8)))) == 70)

The better version adds the duplicate twice - so how do I overcome this inefficiency? I'm sure there's a better way, but I just added a check to stop if first element of C is bigger than first element of b

In [9]:
# when we are adding to c
# if b and c and b[0] > c[0]: return

DEBUG:root:Comparing::[1, 2, 6] and [3, 4, 5]
DEBUG:root:Comparing::[1, 2, 7] and [3, 4, 5]
DEBUG:root:Comparing::[1, 2, 7] and [3, 4, 6]
DEBUG:root:Comparing::[1, 2, 7] and [3, 5, 6]
DEBUG:root:Comparing::[1, 2, 7] and [4, 5, 6]
DEBUG:root:Comparing::[1, 3, 7] and [4, 5, 6]
DEBUG:root:Comparing::[1, 3, 6] and [2, 4, 5]
DEBUG:root:Comparing::[1, 3, 7] and [2, 4, 5]
DEBUG:root:Comparing::[1, 3, 7] and [2, 4, 6]
DEBUG:root:Comparing::[1, 3, 7] and [2, 5, 6]
DEBUG:root:Comparing::[1, 4, 7] and [2, 5, 6]
DEBUG:root:Comparing::[1, 4] and [2, 3]
DEBUG:root:Comparing::[1, 4, 5] and [2, 3, 6]
DEBUG:root:Comparing::[1, 4, 5] and [2, 3, 7]
DEBUG:root:Comparing::[1, 4, 6] and [2, 3, 7]
DEBUG:root:Comparing::[1, 4, 6] and [2, 3, 5]
DEBUG:root:Comparing::[1, 4, 7] and [2, 3, 5]
DEBUG:root:Comparing::[1, 4, 7] and [2, 3, 6]
DEBUG:root:Comparing::[1, 5] and [2, 3]
DEBUG:root:Comparing::[1, 5, 6] and [2, 3, 7]
DEBUG:root:Comparing::[1, 5, 7] and [2, 3, 6]
DEBUG:root:Comparing::[1, 6] and [2, 3]
DEBUG:

70
